<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Регрессия, классификация и кластеризация </b> и <span style="font-weight:bold; color:green">Spark</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru)</span></div>

<a name="0"></a>
<div><span style="font-size:14pt; font-weight:bold">Содержание</span>
    <ol>
        <li><a href="#1">Регрессия</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#1a">Линейная регрессия</a></li>
                <li><a href="#1b">Решающее дерево</a></li>
                <li><a href="#1c">Полимиальная регрессия</a></li>
            </ol>
        </li>
        <li><a href="#2">Классификация</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#2a">Логистическая регрессия</a></li>
                <li><a href="#2b">Решающее дерево</a></li>
            </ol>
        </li>
        <li><a href="#3">Кластеризация</a>
            <ol style = "list-style-type:lower-alpha">
                <li><a href="#3a">K-Mean</a></li>
            </ol>
        </li>
        <li><a href="#4">Источники</a>
        </li>
    </ol>
</div>

<p>[ОПЦИОНАЛЬНО] <b>Настройка среды</b></p>

In [ ]:
import os
import sys

os.environ["SPARK_HOME"]="/opt/cloudera/parcels/SPARK2/lib/spark2"
os.environ["PYSPARK_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

<p>Запуск Spark Context</p>

In [ ]:
import pyspark

In [ ]:
conf = pyspark.SparkConf() \
        .setAppName("mllibRDDApp") \
        .setMaster("yarn") \
        .set("spark.submit.deployMode", "client")

In [ ]:
sc = pyspark.SparkContext(conf=conf)

<p>Дополнительные пакеты</p>

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.colors import ListedColormap
clrMap = ListedColormap(["blue", "red", "green"])

<a name="1"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">1. Регрессия</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "1a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Линейная регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

<p>Загрузите данные из файла</p>

<p>[ЕСЛИ НЕОБХОДИМО] Скопируйте локальный датасет в HDFS</p>

In [ ]:
!hdfs dfs -copyFromLocal data/ml-basics-data/regression_demo.csv data/spark_mllib_rdd

In [ ]:
file_path_reg = "data/spark_mllib_rdd/regression_demo.csv"

In [ ]:
data_rdd = sc.textFile(file_path_reg)
data_rdd.take(5)

<p>Создайте RDD без первой строки (она содержит заголовки)</p>

In [ ]:
def getLabeledPoint(line):
    if line[0] != "X":
        els = [float(el) for el in line.split(",")] 
        return LabeledPoint(els[1], els[0:1])

In [ ]:
data_xy_rdd = data_rdd.map(getLabeledPoint).filter(lambda x: x != None)
data_xy_rdd.take(5)

<p>Отобразите исходных данных</p>

In [ ]:
data_sample = data_xy_rdd.takeSample(withReplacement=False, num=80, seed=123)

In [ ]:
for el in data_sample:
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)
plt.title("Initial Data")
plt.xlabel("X")
plt.ylabel("Y")
plt.show()

<p>Сформируйте обучающее и тестовое подмножества</p>

In [ ]:
train_rdd, test_rdd = data_xy_rdd.randomSplit([0.7, 0.3], seed=123)
train_rdd.persist(), test_rdd.persist()

<p>Отобразите количество элементов в каждом из подмножеств</p>

In [ ]:
train_rdd.count(), test_rdd.count()

<p>Обучите модель линейной регрессии на обучающем подмножестве</p>

<a href="https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionWithSGD">LinearRegressionWithSGD</a>

In [ ]:
model_linReg = LinearRegressionWithSGD.train(train_rdd, intercept=True)
model_linReg

<p>Проверьте результат обучения на тестовом подмножестве с использованием MSE</p>

In [ ]:
pairsTruePred_rdd = test_rdd.map(lambda x: (x.label, model_linReg.predict(x.features)))
pairsTruePred_rdd.take(5)

In [ ]:
mse = pairsTruePred_rdd.map(lambda x: (x[0]-x[1])**2).reduce(lambda x1, x2: x1+x2) / test_rdd.count()
mse

<p>Постройте итоговые графики</p>

In [ ]:
xx = np.arange(0, 5, 0.01)[np.newaxis,:]

# Предсказанные моделью значения
y_pred = model_linReg.predict(xx)

# График 
plt.figure(1)
plt.subplot(1,1,1)

plt.plot(xx[0,:], y_pred, c="black", label="", linewidth=2)
plt.title("Initial Data")
plt.xlabel("X")
plt.ylabel("Y")
for el in data_sample:
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)

plt.show()

<a name = "1b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Решающее дерево
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1a">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#1c">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel

<p>Построй решающее дерево для задачи регрессии с использованием данных из предыдущего примера</p>

<a href="https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.tree.DecisionTree">DecisionTree</a>

In [ ]:
model_decTree = DecisionTree.trainRegressor(train_rdd, impurity="variance", maxDepth=2, categoricalFeaturesInfo={})
model_decTree

<p>Проверьте на тестовом подмножестве</p>

In [ ]:
test_pred_rdd = model_decTree.predict(test_rdd.map(lambda x: x.features))

In [ ]:
pairsTruePred_rdd = test_rdd.map(lambda x: x.label).zip(test_pred_rdd)
pairsTruePred_rdd.take(5)

In [ ]:
mse = pairsTruePred_rdd.map(lambda x: (x[0]-x[1])**2).reduce(lambda x1, x2: x1+x2) / test_rdd.count()
mse

<p>Постройте итоговые графики</p>

In [ ]:
xx = np.arange(0, 5, 0.01)

# Предсказанные моделью значения
y_pred = [model_decTree.predict([el]) for el in xx]

# График
plt.figure(1)
plt.subplot(1,1,1)
plt.plot(xx, y_pred, c="black", label="", linewidth=2)
plt.title("Initial Data")
plt.xlabel("X")
plt.ylabel("Y")
for el in data_sample:
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)

plt.show()

<a name = "1c"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            c. Полимиальная регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#1b">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.feature import StandardScaler

<p>Преобразуйте исходные данные из предыдущих примеров в массив степеней Х</p>

In [ ]:
# степень полинома
deg = 10

# преобразование х в массив х^n, где 0 <= n <= deg
def get1DPolynomialFeatures(x, degree=deg):
    return [x**i for i in range(0, degree+1)]

# RDD транформация 
train_features_poly_rdd = train_rdd.map(lambda x: get1DPolynomialFeatures(x.features[0], deg))
test_features_poly_rdd = test_rdd.map(lambda x: get1DPolynomialFeatures(x.features[0], deg))

# вывод первых пяти элементов RDD для обучающих данных
train_features_poly_rdd.take(5)

<p>Нормализуйте каждый полученный признак к стандартному нормальному распределнию - стандартизация</p>

In [ ]:
# модель нормализации
norm = StandardScaler(withMean=True, withStd=True)

# вычисление математического ожидания и стандартного отклонения
scaler = norm.fit(train_features_poly_rdd)

# нормализация train_features_poly_rdd и test_features_poly_norm_rdd
train_features_poly_norm_rdd = scaler.transform(train_features_poly_rdd)
test_features_poly_norm_rdd = scaler.transform(test_features_poly_rdd)

# вывод первых пяти элементов RDD для обучающих данных
train_features_poly_norm_rdd.take(5)

<p>Создайте новые обучающий и тестовый наборы с учетом проведенных преобразований с Х</p>

In [ ]:
# cформируйте RDD с действиетельными значениями
train_labels_rdd = train_rdd.map(lambda x: x.label)
test_labels_rdd = test_rdd.map(lambda x: x.label)

# объедините train_labels_rdd и train_features_poly_norm_rdd и поместите значения в LabeledPoint
train_poly_rdd = train_labels_rdd.zip(train_features_poly_norm_rdd).map(lambda x: LabeledPoint(x[0], x[1]))

# объедините test_labels_rdd и test_features_poly_norm_rdd и поместите значения в LabeledPoint
test_poly_rdd = test_labels_rdd.zip(test_features_poly_norm_rdd).map(lambda x: LabeledPoint(x[0], x[1]))

# вывод первых пяти элементов RDD для обучающих данных
train_poly_rdd.take(5)

<p>Произведите обучение модели линейного регрессии</p>

<a href="https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.regression.LinearRegressionWithSGD">LinearRegressionWithSGD</a>

In [ ]:
model_linReg = LinearRegressionWithSGD.train(train_poly_rdd, 
                                             intercept=True, 
                                             regParam=10e-4, 
                                             regType="l2", 
                                             iterations=300)
model_linReg

<p>Проверьте на тестовом подмножестве</p>

In [ ]:
pairsTruePred_rdd = test_poly_rdd.map(lambda x: (x.label, model_linReg.predict(x.features)))
pairsTruePred_rdd.take(5)

In [ ]:
mse = pairsTruePred_rdd.map(lambda x: (x[0]-x[1])**2).reduce(lambda x1, x2: x1+x2) / test_rdd.count()
mse

<p>Постройте итоговые графики</p>

In [ ]:
xx = np.arange(0, 5, 0.01)[np.newaxis,:]

# Предсказанные моделью значения
xx_powers_rdd = sc.parallelize(list(map(get1DPolynomialFeatures, np.arange(0, 5, 0.01))))
xx_scaled_rdd = scaler.transform(xx_powers_rdd)
y_pred_rdd = xx_scaled_rdd.map(lambda x: model_linReg.predict(x))
y_pred = y_pred_rdd.collect()

# График
plt.figure(1)
plt.subplot(1,1,1)
plt.plot(xx[0,:], y_pred, c="black", label="", linewidth=2)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
for el in data_sample:
    plt.plot(el.features[0], el.label, "bo")
plt.grid(True)

plt.show()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Классификация</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "2a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. Логистическая регрессия
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithSGD

<p>Загрузите данные из файла</p>

<p>[ЕСЛИ НЕОБХОДИМО] Скопируйте локальный датасет в HDFS</p>

In [ ]:
!hdfs dfs -copyFromLocal data/ml-basics-data/classification_demo.csv data/spark_mllib_rdd

In [ ]:
data_path_class = "data/spark_mllib_rdd/classification_demo.csv"

In [ ]:
data_rdd = sc.textFile(data_path_class)

<p>Создайте RDD без первой строки (она содержит заголовки)</p>

In [ ]:
def getLabeledPoint(line):
    if line[0]!="X":
        els = [float(el) for el in line.split(",")]
        return LabeledPoint(els[2], els[:2])
    
data_xy_rdd = data_rdd.map(getLabeledPoint).filter(lambda x: x!= None)
data_xy_rdd.take(5)

<p>Отобразите исходных данных</p>

In [ ]:
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
for el in data_xy_rdd.takeSample(withReplacement=False, num=80, seed=123):
    plt.plot(el.features[0], el.features[1], "o", color=clrMap.colors[int(el.label)])
plt.grid(True)
plt.show()

<p>Сформируйте обучающее и тестовое подмножества</p>

In [ ]:
train_rdd, test_rdd = data_xy_rdd.randomSplit([0.7, 0.3], seed=123)
train_rdd.persist(), test_rdd.persist()

<p>Обучите модель логистической регрессии на обучающем подмножестве</p>

<a href='https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.LogisticRegressionWithSGD'>LogisticRegressionWithSGD</a>

In [ ]:
model_logReg = LogisticRegressionWithSGD.train(train_rdd, intercept=True)
model_logReg

<p>Проверьте результат обучения на тестовом подмножестве с использованием Accuracy</p>

In [ ]:
pairsTruePred_rdd = test_rdd.map(lambda x: (x.label, model_logReg.predict(x.features)))
pairsTruePred_rdd.take(5)

In [ ]:
accuracy = pairsTruePred_rdd.filter(lambda x: x[0] == x[1]).count() / test_rdd.count()
accuracy

<p>Постройте итоговые графики</p>

In [ ]:
train_data_sample = train_rdd.takeSample(withReplacement=False, num=80, seed=123)
test_data_sample = test_rdd.takeSample(withReplacement=False, num=80, seed=123)

In [ ]:
step = 0.1
xx, yy = np.meshgrid(np.arange(-2, 14, step), np.arange(-4, 12, step))
points = np.c_[xx.ravel(), yy.ravel()]
Z = np.array(list(map(model_logReg.predict, points)))
Z = Z.reshape(xx.shape)

plt.figure(1, figsize=[12, 4])

plt.subplot(1,2,1)
plt.title("Train data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)

for el in train_data_sample:
    plt.plot(el.features[0], el.features[1], "o", 
             color=clrMap.colors[int(el.label)], markersize=8, alpha=0.5)
    plt.plot(el.features[0], el.features[1], "o", 
             color=clrMap.colors[int(model_logReg.predict(el.features))], markersize=4)
    
plt.grid(True)

plt.subplot(1,2,2)
plt.title("Test data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.contourf(xx, yy, Z, cmap=clrMap, alpha=.5)

for el in test_data_sample:
    plt.plot(el.features[0], el.features[1], "o", 
             color=clrMap.colors[int(el.label)], markersize=8, alpha=0.5)
    plt.plot(el.features[0], el.features[1], "o", 
             color=clrMap.colors[int(model_logReg.predict(el.features))], markersize=4)
plt.grid(True)

plt.show()

<p>Сохраните модель</p>

In [ ]:
model_logReg.save(sc, "logReg")
#model_logReg_reload = LogisticRegressionModel.load(sc, "logReg")

<a name = "2b"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            b. Решающее дерево
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#2a">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#2c">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
# TODO

<a name="3"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">3. Кластеризация</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a name = "3a"></a>
<div style = "display:table; width:100%">
    <div style = "display:table-row">
        <div style = "display:table-cell; width:80%; font-style:italic; font-weight:bold; font-size:12pt">
            a. K-Means
        </div>
        <div style="display:table-cell; border:1px solid lightgrey; width:20%">
            <div style = "display:table-cell; width:10%; text-align:center; background-color:whitesmoke;">
                <a href="#3">Назад</a>
            </div>
            <div style = "display:table-cell; width:10%; text-align:center;">
                <a href="#3b">Далее</a>
            </div>
        </div>
    </div>
</div>

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel

<p>Загрузите исходные данные из файла</p>

<p>[ЕСЛИ НЕОБХОДИМО] Скопируйте локальный датасет в HDFS</p>

In [ ]:
!hdfs dfs -copyFromLocal data/ml-basics-data/clusters_demo.csv data/spark_mllib_rdd

In [ ]:
data_path_cluster = "data/spark_mllib_rdd/clusters_demo.csv"

In [ ]:
data_rdd = sc.textFile(data_path_cluster)

def getArray(x):
    return np.array(list(map(float, x.split(","))))

data_arr_rdd = data_rdd.map(lambda x: getArray(x))
data_arr_rdd.persist()
data_arr_rdd.take(5)

<p>Отобразите исходных данных</p>

In [ ]:
for el in data_arr_rdd.takeSample(withReplacement=False, num=80, seed=123):
    plt.plot(el[0], el[1], "bo")
plt.grid(True)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

<p>Кластеризуйте данные K-means методом</p>

<a href="https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.clustering.KMeans">KMeans</a>

In [ ]:
clusters = KMeans.train(data_arr_rdd, k=3, maxIterations=10, initializationMode="k-means||")
clusters

In [ ]:
result_rdd = data_arr_rdd.map(lambda x: clusters.predict(x)).zip(data_arr_rdd)
result_rdd.take(5)

<p>Постройте итоговые графики</p>

In [ ]:
for el in result_rdd.takeSample(withReplacement=False, num=80, seed=123):
    plt.plot(el[1][0], el[1][1], "o", color=clrMap.colors[el[0]])
        
plt.grid(True)
plt.title("Initial Data")
plt.xlabel("X0")
plt.ylabel("X1")
plt.show()

<p>Сохраните модель кластеризации</p>

In [ ]:
clusters.save(sc, "clusterModel")
#sameModel = KMeansModel.load(sc, "clusterModel")

<a name="4"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">4. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="https://spark.apache.org/docs/2.2.0/mllib-guide.html">MLlib: RDD-based API</a>